# WHY FAKE NEWS IS A PROBLEM?
**Fake news refers to misinformation, disinformation or mal-information which is spread through word of mouth and traditional media and more recently through digital forms of communication such as edited videos, memes, unverified advertisements and social media propagated rumours.Fake news spread through social media has become a serious problem, with the potential of it resulting in mob violence, suicides etc as a result of misinformation circulated on social media.**
![image.png](attachment:image.png)

# BRIEF DESCRIPTION OF DATASET
**This dataset consists of about 40000 articles consisting of fake as well as real news. Our aim is train our model so that it can correctly predict whether a given piece of news is real or fake.The fake and real news data is given in two separate datasets with each dataset consisting around 20000 articles each.**

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!ls
%cd /content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset

drive  sample_data
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


In [4]:
%cd fake-and-real-news-dataset/

[Errno 2] No such file or directory: 'fake-and-real-news-dataset/'
/content/drive/My Drive/Colab Notebooks/projet_sopra/input/fake-and-real-news-dataset


# LOADING THE NECESSARY LIBRARIES

In [5]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf

# IMPORTING THE DATASET

# DATA VISUALIZATION AND PREPROCESSING

**Import data in tf dataset**

In [6]:
true_dataset = tf.data.experimental.CsvDataset(filenames = ['True.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)
false_dataset = tf.data.experimental.CsvDataset(filenames = ['Fake.csv'], record_defaults = [tf.string, tf.string], select_cols=[0,1], header=True)

In [7]:
for idx, row in enumerate(true_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b'As U.S. budget fight looms, Republicans flip their fiscal script'>, <tf.Tensor: shape=(), dtype=string, numpy=b'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a \xe2\x80\x9cfiscal conservative\xe2\x80\x9d on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS\xe2\x80\x99 \xe2\x80\x9cFace the Nation,\xe2\x80\x9d drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. Pr

**Join text and title**

In [8]:
def preprocess_join_sentence(s1, s2):
    ret = tf.strings.reduce_join([s1, s2],separator=" ", axis=-1)
    return ret

true_dataset = true_dataset.map(preprocess_join_sentence)
false_dataset = false_dataset.map(preprocess_join_sentence)

In [9]:
for idx, row in enumerate(true_dataset):
    print (row)
    if idx > 3:
        break

tf.Tensor(b'As U.S. budget fight looms, Republicans flip their fiscal script WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a \xe2\x80\x9cfiscal conservative\xe2\x80\x9d on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS\xe2\x80\x99 \xe2\x80\x9cFace the Nation,\xe2\x80\x9d drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military 

### Add labels to dataset

In [10]:
cat_true = tf.data.Dataset.from_tensor_slices(np.ones(len(list(true_dataset)), dtype=int))
cat_false = tf.data.Dataset.from_tensor_slices(np.zeros(len(list(false_dataset)), dtype=int))

In [11]:
true_dataset = tf.data.Dataset.zip((true_dataset, cat_true))
false_dataset = tf.data.Dataset.zip((false_dataset, cat_false))

In [12]:
full_dataset = false_dataset.concatenate(true_dataset)

In [13]:
for idx, row in enumerate(full_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b' Donald Trump Sends Out Embarrassing New Year\xe2\x80\x99s Eve Message; This is Disturbing Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What k

**Formate sentences**

In [14]:
from nltk.corpus import stopwords
nltk.download('stopwords')

def rm_stopwords(text):
  stop = set(stopwords.words('english'))
  nltk.download('punkt')

  stop = list(stop)
  ret = tf.strings.strip(text)

  for i in stop:
    ret = tf.strings.regex_replace(ret, '\\b'+i+'\\b', '')
  # Remove adding spaces

  ret = tf.strings.regex_replace(ret, ' +', ' ')
  return ret

def preprocess_sentence(sentence, label):
  ret = tf.strings.lower(sentence) # to lower

  ret = tf.strings.strip(ret) # create list of words 

  # Denoise text
  ret = tf.strings.regex_replace(ret, r'[^\x00-\x7f]','') # Removing adding chars to decode in UTF-8
  ret = tf.strings.regex_replace(ret, '<[^<]+?>', '') # Removing HTML chars
  ret = tf.strings.regex_replace(ret, '\[[^]]*\]', '') # Removing the square brackets
  ret = tf.strings.regex_replace(ret, 'http\S+', '') # Removing URL's

  ret = tf.strings.regex_replace(ret, '!"#$%&(),-./:;<=>?@\t\n', '') # Keras tokenizer filter
  ret = tf.strings.regex_replace(ret, '[^A-Za-z0-9]+', ' ') # remove ponctuation

  ret = rm_stopwords(ret) # Removing stopwords

  return ret, label

full_dataset = full_dataset.map(preprocess_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: while/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [15]:
for idx, row in enumerate(full_dataset):
    print (row)
    if idx > 3:
        break

(<tf.Tensor: shape=(), dtype=string, numpy=b'donald trump sends embarrassing new years eve message disturbing donald trump wish americans happy new year leave instead give shout enemies haters dishonest fake news media former reality show star one job country rapidly grows stronger smarter want wish friends supporters enemies haters even dishonest fake news media happy healthy new year president angry pants tweeted 2018 great year america country rapidly grows stronger smarter want wish friends supporters enemies haters even dishonest fake news media happy healthy new year 2018 great year america donald j trump realdonaldtrump december 31 2017trump tweet went welll expect kind president sends new year greeting like despicable petty infantile gibberish trump lack decency even allow rise gutter long enough wish american citizens happy new year bishop talbert swan talbertswan december 31 2017no one likes calvin calvinstowell december 31 2017your impeachment would make 2018 great year amer

In [16]:
!pip install -q tensorflow-text


     |████████████████████████████████| 3.4MB 11.1MB/s 


### Glove

**Count max occurences in dataset**

In [17]:
from collections import Counter

def freq_words(inputs, max_size=15000): # input dataset, returns list
  count = Counter()

  numpy_sentences = []
  for sentences, labels in full_dataset.take(-1):  # take all elements in dataset
      numpy_sentences.extend(tf.strings.split(sentences).numpy())
      numpy_labels = labels.numpy()
  list_words = numpy_sentences

  for word in list_words:
    # print(word)
    count[word] += 1
  result = [word for word, num in count.most_common(max_size)]
  return result

max_freq_words = freq_words(full_dataset)

In [18]:
max_freq_words = [w.decode("utf-8") for w in max_freq_words]

In [19]:
max_freq_words[:10]

['trump',
 'said',
 'u',
 'president',
 'would',
 'people',
 'one',
 'state',
 'new',
 'also']

**Find this occurences in the pretrained glove file**

In [20]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Build vocabulary and relevent vector
words = []
# Adding 2 empty vector of dimension 100 which will be used in padding and unknown token(word)
# According to previous knowledge i have idea that it contains 400,000 words
# Added extra 2 rows because when we will use inside TextVectorization add blank i.e. '' and ['UNK'] inside vocabulary

counter = 0

vectors = np.zeros((10000, 100))

with open(f"../glove-twitter/glove.twitter.27B.100d.txt") as f:
    lines = f.readlines()

    for idx, line in enumerate(lines):
      if counter >= 10000-2:
        break

      split_line = line.split()
      
      if split_line[0] in max_freq_words:
        words.append(split_line[0])
        vectors[counter+2] = split_line[1:]
        counter+=1

print(f"Number of words are {len(words)}")
print(f"Shape of vector is {vectors.shape}")

Number of words are 9998
Shape of vector is (10000, 100)


In [21]:
vectors

array([[ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.      ,  0.      ,  0.      , ...,  0.      ,  0.      ,
         0.      ],
       [ 0.6047  ,  0.89542 ,  0.27923 , ...,  0.51566 ,  0.84726 ,
         0.020946],
       ...,
       [ 0.12175 ,  0.028571,  0.92407 , ..., -0.37332 , -0.50388 ,
         0.09708 ],
       [-0.13091 ,  0.37478 ,  0.53096 , ..., -0.51894 , -0.21917 ,
         0.48503 ],
       [ 0.49506 , -0.28023 , -0.10678 , ..., -0.19052 , -0.010599,
        -0.36815 ]])

In [22]:
words[1120:1130]

['sir',
 'math',
 'moving',
 'losing',
 'nights',
 'rap',
 'sam',
 'march',
 'truly',
 'killing']

In [23]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# TextVectorization hyper parameter
max_vocab = 10000
max_len = 300
vectorize_layer_glove = TextVectorization(max_tokens=max_vocab,
                                                    standardize="lower_and_strip_punctuation",
                                                    split="whitespace",
                                                    output_mode="int",
                                                    output_sequence_length=max_len,
                                                    vocabulary=words)


# Vocabulary set into layer
vectorize_layer_glove.set_vocabulary(words)
print(f"Top 10 words {vectorize_layer_glove.get_vocabulary()[:10]}")
print(f"Some words {vectorize_layer_glove.get_vocabulary()[-1010:-1000]} \n")

Top 10 words ['', '[UNK]', 'rt', 'de', 'n', 'la', 'e', 'u', 'en', 'el']
Some words ['aspen', 'hsbc', 'assigned', 'relying', 'plc', 'bailout', 'complained', 'opposing', 'interfere', 'accompany'] 



In [24]:
def int_vectorize_text(text, label):
  text = tf.expand_dims(text, -1)
  return vectorize_layer_glove(text), label

train_data_tokenized = full_dataset.map(int_vectorize_text)

In [25]:
model_glove = tf.keras.models.Sequential()
model_glove.add(tf.keras.layers.Input(shape=(1,), dtype=tf.string))
model_glove.add(vectorize_layer_glove)
model_glove.predict(["expect terror fbi steps use isis stings 21st century wire says real terrorist threat thing ..."])

array([[1009, 2933, 4447, 2580,  312, 7691,    1,    1, 3899, 4622,  353,
         111, 4894, 4184,  162,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0, 

In [26]:
for idx, row in enumerate(train_data_tokenized):
  if idx == 1:
    print(row)
    break

(<tf.Tensor: shape=(1, 300), dtype=int64, numpy=
array([[ 662, 6713, 6659,    1,  572, 3190,    1, 5838,  221, 3866, 5393,
        6172, 6267, 9655,   70,  137,   31, 9360,   10,  229,  102, 4496,
        9298, 8162,    1, 2446, 5838,    1, 4275, 2150, 4447,  813, 2169,
        6659,  782, 8162,  572, 5838, 2303, 7134,    1,   30, 1002,  324,
        2447, 6659, 2367, 9372, 1476,    1,  662, 1407,  871, 4469, 2776,
        3190, 5412, 2315, 5871, 4248,  254,    1,    1,  251, 6659, 5739,
        7002,   64, 6665, 1847, 1217,    1, 6349, 1328, 1407,  871,  873,
         310,  205, 7308,   30, 1002, 1165, 6659, 1145,    1,    1,    9,
        2108, 2545,  236,  746, 3101, 1561, 2447, 6857,  473, 5580,  144,
        3260, 6659,  109, 6659, 1480, 6645,  873,  251,  214,    1,    1,
        4469, 3802, 9846, 4685,    1, 3190, 4665,  726,  240, 1047, 3652,
        6624, 4741, 8305, 5871, 4248,  732,   64, 1162,    1,  159,   75,
           1, 1407, 4608, 3802, 4685,    1,    1, 1226, 2195,  

In [27]:
len(vectors[0])

100

### Prepare data to train

In [28]:
# Model params
batch_size = 256
epochs = 10
embed_size = 100
maxlen = 300
max_features = 10000

**Model**

In [29]:
#Defining Neural Network
model = Sequential()
#Non-trainable embeddidng layer
model.add(Embedding(max_features, output_dim=embed_size, weights=[vectors], input_length=maxlen, trainable=False))
#LSTM 
model.add(LSTM(units=128 , return_sequences = True))
model.add(LSTM(units=64))
model.add(Dense(units = 32 , activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.00001)


In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 100)          1000000   
_________________________________________________________________
lstm (LSTM)                  (None, 300, 128)          117248    
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                49408     
_________________________________________________________________
dense (Dense)                (None, 32)                2080      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,168,769
Trainable params: 168,769
Non-trainable params: 1,000,000
_________________________________________________________________


### TODO : add validation dataset and shuffle data

In [31]:
full_dataset = train_data_tokenized.map(lambda x_text, x_label: (x_text, tf.expand_dims(x_label, -1)))

In [32]:
len(list(full_dataset))

44898

In [33]:
DATASET_SIZE = len(list(full_dataset))

train_size = int(0.8 * DATASET_SIZE)
test_size = int(0.2 * DATASET_SIZE)

full_dataset = full_dataset.shuffle(buffer_size=DATASET_SIZE, seed=42)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)

In [39]:
for idx, row in enumerate(train_dataset):
  if idx == 1:
    print(row)
    break

(<tf.Tensor: shape=(1, 300), dtype=int64, numpy=
array([[ 484, 4508,  291, 6659,    1,  138,  207, 4173, 2274, 2864, 3384,
        3008, 7256, 9662,    1,   98,   64,  174,  484, 4508,  291, 6659,
        3152, 3384, 2864,  187, 1480, 1145, 2362,    1, 1922, 4398, 6891,
          41,  551,   98, 5308, 7745,  229,  273,  526, 7745,    1, 8986,
           1,  273,  526,  229,   33,  214,  265, 1972,    1,  273,  604,
        9478, 4597, 4508, 6659, 8143, 1828,   60,   32, 1145,  861, 2195,
        2222,  719,  311, 1483,    1,    1, 1242,  144,    1, 6430, 2447,
           1,    1,    1, 8986,    1, 6430, 4544, 8115, 1228, 1342,  431,
           1,  432,    1, 8986,    1,   29,    1, 5220, 3972, 1833,    1,
         846,  795,  170, 8592,  120, 2609, 4597, 4508, 6659,    1, 5372,
          91,   29,    1, 8986,  159,    1, 1471,  170,  585,  271,   93,
          29,   64,   10,   41, 6659,   91, 2372, 6704,    1,  273,  207,
          21,  120,   33,  120,  166,   92, 1003,  523, 1479, 7

In [35]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18023635697200740924
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11153436864
locality {
  bus_id: 1
  links {
  }
}
incarnation: 18430811517388179845
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"
]


In [ ]:
# Train for 2 epochs
model.fit(train_dataset, batch_size = batch_size, validation_data=test_dataset, epochs=2, steps_per_epoch=20)